In [1]:
import os
import sys
initial_path = set(sys.path)
CARLA_EGG_PATH = "/home/rudy/Documents/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg"
sys.path.append(CARLA_EGG_PATH)

# ADD 
try:
    sys.path.append(os.path.abspath('.') + '/PythonAPI/carla')
except IndexError:
    pass

new_paths = set(sys.path) - initial_path
for path in new_paths:
    print(f"Added: {path} to the Path")


Added: /home/rudy/Documents/carla-dataset-runner/playground/PythonAPI/carla to the Path
Added: /home/rudy/Documents/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg to the Path


In [2]:
import carla 
import os 
import random
import numpy as np


In [3]:
try:
    import queue
except ImportError:
    import Queue as queue


class CarlaSyncMode(object):
    """
    Context manager to synchronize output from different sensors. Synchronous
    mode is enabled as long as we are inside this context

        with CarlaSyncMode(world, sensors) as sync_mode:
            while True:
                data = sync_mode.tick(timeout=1.0)

    """

    def __init__(self, world, *sensors, **kwargs):
        self.world = world
        self.sensors = sensors
        self.frame = None
        self.delta_seconds = 1.0 / kwargs.get('fps', 20)
        self._queues = []
        self._settings = None

    def __enter__(self):
        self._settings = self.world.get_settings()
        self.frame = self.world.apply_settings(carla.WorldSettings(
            no_rendering_mode=False,
            synchronous_mode=True,
            fixed_delta_seconds=self.delta_seconds))

        def make_queue(register_event):
            q = queue.Queue()
            register_event(q.put)
            self._queues.append(q)

        make_queue(self.world.on_tick)
        for sensor in self.sensors:
            make_queue(sensor.listen)
        return self

    def tick(self, timeout):
        self.frame = self.world.tick()
        data = [self._retrieve_data(q, timeout) for q in self._queues]
        assert all(x.frame == self.frame for x in data)
        return data

    def __exit__(self, *args, **kwargs):
        self.world.apply_settings(self._settings)

    def _retrieve_data(self, sensor_queue, timeout):
        while True:
            data = sensor_queue.get(timeout=timeout)
            if data.frame == self.frame:
                return data

    def tick_no_data(self):
        self.frame = self.world.tick()

In [4]:

client = carla.Client('127.0.0.1', 2000) 
client.set_timeout(30.0) 
world = client.load_world('Town02') 
settings = world.get_settings() 
settings.fixed_delta_seconds = 0.1 
settings.synchronous_mode = True 
world.apply_settings(settings) 
blueprint_library = world.get_blueprint_library() 
traffic_manager = client.get_trafficmanager(8000) 
traffic_manager.set_hybrid_physics_mode(True)
vehicle_list = [] 


In [12]:
transform = random.choice(world.get_map().get_spawn_points()) 
vehicle = world.try_spawn_actor(random.choice(blueprint_library.filter("vehicle.*")) , transform) 
print("Vehicle spawned!")

vehicle_list.append(vehicle) 
vehicle.set_autopilot(True, traffic_manager.get_port()) 

Vehicle spawned!


In [15]:
sensor_width = 800
sensor_height = 600
fov = 110
bp = blueprint_library.find('sensor.camera.rgb')
# bp.set_attribute('enable_postprocess_effects', 'True')  # https://carla.readthedocs.io/en/latest/bp_library/
bp.set_attribute('image_size_x', f'{sensor_width}')
bp.set_attribute('image_size_y', f'{sensor_height}')
bp.set_attribute('fov', f'{fov}')

# Adjust sensor relative position to the vehicle
spawn_point = carla.Transform(carla.Location(x=1.0, z=2.0))
rgb_camera = world.spawn_actor(bp, spawn_point, attach_to=vehicle)
rgb_camera.blur_amount = 0.0
rgb_camera.motion_blur_intensity = 0
rgb_camera.motion_max_distortion = 0

# Camera calibration
calibration = np.identity(3)
calibration[0, 2] = sensor_width / 2.0
calibration[1, 2] = sensor_height / 2.0
calibration[0, 0] = calibration[1, 1] = sensor_width / (2.0 * np.tan(fov * np.pi / 360.0))
rgb_camera.calibration = calibration  # Parameter K of the camera
print("Camera added successfully!")

Camera added successfully!


In [17]:
import cv2
import time

for _ in range(200): 
    world.tick(2.0) 
    location = vehicle.get_location() 
    # velocity = vehicle.get_velocity() 
    # control = vehicle.get_control() 

    # img = np.array(rgb_data.raw_data)
    #img = img.reshape((sensor_height, sensor_width, 4))
    #img = img[:, :, :3]  # taking out opacity channel
    #cv2.imshow('rgb', img)
    #cv2.waitKey(10)



    sys.stdout.write("\r")
    sys.stdout.write(f"[{time.time():.0f}] Location: ({location.x:.2f}, {location.y:.2f}, {location.z:.2f})")
    sys.stdout.flush()

[1617727199] Location: (181.80, 187.92, 0.21)

KeyboardInterrupt: 

In [ ]:
for actor in vehicle_list: 
    actor.destroy() 